# Gemini TTS :

- Gemini API 提供了原生的文字轉語音 (Text-to-Speech, TTS) 功能，可以將文字輸入轉換為單人或多人的語音音訊。此功能具備高度可控性，允許使用者透過自然語言來調整語音的風格、口音、速度和語氣。

#### **核心功能與用途**

- **與 Live API 的區別**：
  - **TTS API**：專為需要精確朗讀文字並進行精細風格控制的場景設計，例如製作 Podcast 或有聲書。
  - **Live API**：專為互動式、非結構化的即時對話場景設計。
- **主要優勢**：可透過自然語言提示詞 (prompt) 精確控制輸出的音訊風格。

細節 : https://ai.google.dev/gemini-api/docs/speech-generation


In [6]:
from google import genai
from google.genai import types
from dotenv import load_dotenv
import os

load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

client = genai.Client(api_key=GOOGLE_API_KEY)

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


In [7]:
from google import genai
from IPython.display import display

In [12]:
# TTS 目前只有在 Google AI Studio
for model in client.models.list(config={"query_base": True}):
    if "tts" in model.name:
        print(model.name)

models/gemini-2.5-flash-preview-tts
models/gemini-2.5-pro-preview-tts


### 單人語音轉換


In [ ]:
import wave

from IPython.display import Audio


# Set up the wave file to save the output:
def wave_file(filename, pcm, channels=1, rate=24000, sample_width=2):
    print("Writing audio file with parameters:")
    print(f"Channels: {channels}")
    print(f"Sample rate: {rate}")
    print(f"Sample width: {sample_width}")
    print(f"Data length: {len(pcm)} bytes")

    with wave.open(filename, "wb") as wf:
        wf.setnchannels(channels)
        wf.setsampwidth(sample_width)
        wf.setframerate(rate)
        wf.writeframes(pcm)

In [ ]:
PROMPT = "以專業的語氣說明: 現在 Gemini 支援 TTS! 並且可以準確地朗讀中文"

VOICE = "Leda"  # 有其他 30 種聲音選擇

client = genai.Client(api_key=GOOGLE_API_KEY)

response = client.models.generate_content(
    model="gemini-2.5-flash-preview-tts",
    contents=PROMPT,
    config=types.GenerateContentConfig(
        response_modalities=["audio"],
        speech_config=types.SpeechConfig(
            voice_config=types.VoiceConfig(
                prebuilt_voice_config=types.PrebuiltVoiceConfig(
                    voice_name=VOICE,
                )
            )
        ),
    ),
)

# Debug the response structure
print("\nResponse structure:")
print(f"Number of candidates: {len(response.candidates)}")
print(f"Content parts: {len(response.candidates[0].content.parts)}")
print(f"Part type: {type(response.candidates[0].content.parts[0])}")

data = response.candidates[0].content.parts[0].inline_data.data

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.



Response structure:
Number of candidates: 1
Content parts: 1
Part type: <class 'google.genai.types.Part'>


In [16]:
response.usage_metadata

GenerateContentResponseUsageMetadata(
  candidates_token_count=146,
  candidates_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.AUDIO: 'AUDIO'>,
      token_count=146
    ),
  ],
  prompt_token_count=23,
  prompt_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=23
    ),
  ],
  total_token_count=169
)

In [ ]:
rate = 24000
file_name = "../audio/single_voice_out.wav"

print(f"\nSaving sample rate: {rate}")
wave_file(file_name, data, rate=rate)


Saving sample rate: 24000
Writing audio file with parameters:
Channels: 1
Sample rate: 24000
Sample width: 2
Data length: 280846 bytes


In [ ]:
audio_file_path = "../audio/single_voice_out.wav"
display(Audio(audio_file_path))

### 包裝成 Function : 以便後續使用


In [20]:
def generate_tts(PROMPT, VOICE, file_name):
    client = genai.Client(api_key=GOOGLE_API_KEY)

    response = client.models.generate_content(
        model="gemini-2.5-flash-preview-tts",
        contents=PROMPT,
        config=types.GenerateContentConfig(
            response_modalities=["audio"],
            speech_config=types.SpeechConfig(
                voice_config=types.VoiceConfig(
                    prebuilt_voice_config=types.PrebuiltVoiceConfig(
                        voice_name=VOICE,
                    )
                )
            ),
        ),
    )

    data = response.candidates[0].content.parts[0].inline_data.data
    # set the sample rate
    rate = 24000
    file_name = f"../audio/{file_name}.wav"

    print(f"\nSaving sample rate: {rate}")
    wave_file(file_name, data, rate=rate)

    return file_name

In [ ]:
PROMPT = "尊敬的客戶你好，目前電話都在忙線中，請稍等"
VOICE = "Gacrux"  # 聲音風格 : Mature
FILENAME = "Gacrux_01"

audio_file_path = generate_tts(PROMPT, VOICE, FILENAME)

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.



Saving sample rate: 24000
Writing audio file with parameters:
Channels: 1
Sample rate: 24000
Sample width: 2
Data length: 244366 bytes


In [22]:
display(Audio(audio_file_path))

In [ ]:
PROMPT = "主持人的語氣 : 歡迎來到這場會議，今天主要重點討論 Gemini 有哪些語音的功能可以提供轉語音進行應用"
VOICE = "Erinome"  # 聲音風格 : Clear
FILENAME = "Erinome_01"

audio_file_path = generate_tts(PROMPT, VOICE, FILENAME)

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.



Saving sample rate: 24000
Writing audio file with parameters:
Channels: 1
Sample rate: 24000
Sample width: 2
Data length: 407566 bytes


In [27]:
display(Audio(audio_file_path))

In [ ]:
PROMPT = "Say cheerfully : 通過 Gemini TTS 我們甚至可以生成與 NotebookLM Audio 一樣的 Podcast"
VOICE = "Charon"  # 聲音風格 : Informative
FILENAME = "charon_02"

audio_file_path = generate_tts(PROMPT, VOICE, FILENAME)

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.



Saving sample rate: 24000
Writing audio file with parameters:
Channels: 1
Sample rate: 24000
Sample width: 2
Data length: 344206 bytes


In [29]:
display(Audio(audio_file_path))

### 多人語音生成


In [ ]:
transcript = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="""請生成一篇約200字的精簡教學對話。

**主題是：特斯拉超級充電站 - 核心操作速成**

內容聚焦在一位講師指導一位新車主，快速完成特斯拉超級充電站的插拔核心步驟。省略所有閒聊與次要資訊。

**角色與語氣：**

* **講師:** 語氣專業，指令清晰簡潔。
* **新車主:** 專注於提問關鍵操作，例如「怎麼插？」、「怎麼拔？」。

**對話必須包含以下兩個核心教學點：**

1.  **開始充電：**
    * 指導拿起充電槍，按住頂端按鈕來打開車輛的充電埠。
    * 確認插入後，講師提醒：「看到燈號變綠色閃爍，就代表成功了。」

2.  **結束充電：**
    * 指導先在車內螢幕上點擊「停止充電」。
    * 提醒拔出充電槍的訣竅：「長按槍上按鈕，等燈號變白，就能輕鬆拔出來了。」

**目標：** 對話要簡潔、直接，讓新手只看一次就能明白最重要的「插上」與「拔除」這兩個動作。""",
).text

print(f"Transcript: {transcript}")

Transcript: **主題：特斯拉超級充電站 - 核心操作速成**

**角色：**
*   **講師：** 語氣專業，指令清晰簡潔。
*   **新車主：** 專注於提問關鍵操作。

---

**講師：** 好，超級充電核心操作。我們從插入開始。

**新車主：** 怎麼插？

**講師：** 拿起充電槍，按住槍頂端的按鈕。車輛充電埠會自動打開。

**新車主：** （操作中）喔，開了。

**講師：** 直接插入。看到燈號變綠色閃爍，就代表成功了。

**新車主：** 了解。那怎麼拔？

**講師：** 先在車內螢幕上點擊「停止充電」。

**新車主：** （操作中）好了。

**講師：** 然後，長按槍上的按鈕，等燈號變白，就能輕鬆拔出來了。記住，等燈號變白。

**新車主：** 明白了！

**講師：** 這兩點掌握即可。


In [ ]:
response = client.models.generate_content(
    model="gemini-2.5-flash-preview-tts",
    contents=transcript,
    config=types.GenerateContentConfig(
        response_modalities=["AUDIO"],
        speech_config=types.SpeechConfig(
            multi_speaker_voice_config=types.MultiSpeakerVoiceConfig(
                speaker_voice_configs=[
                    types.SpeakerVoiceConfig(
                        speaker="新車主",
                        voice_config=types.VoiceConfig(
                            prebuilt_voice_config=types.PrebuiltVoiceConfig(
                                voice_name="Zephyr",
                            )
                        ),
                    ),
                    types.SpeakerVoiceConfig(
                        speaker="講師",
                        voice_config=types.VoiceConfig(
                            prebuilt_voice_config=types.PrebuiltVoiceConfig(
                                voice_name="Puck",
                            )
                        ),
                    ),
                ]
            )
        ),
    ),
)

data = response.candidates[0].content.parts[0].inline_data.data

In [32]:
response.usage_metadata

GenerateContentResponseUsageMetadata(
  candidates_token_count=852,
  candidates_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.AUDIO: 'AUDIO'>,
      token_count=852
    ),
  ],
  prompt_token_count=250,
  prompt_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=250
    ),
  ],
  total_token_count=1102
)

In [ ]:
# set the sample rate
rate = 24000
file_name = "../audio/multi_01.wav"

print(f"\nSaving sample rate: {rate}")
wave_file(file_name, data, rate=rate)


Saving sample rate: 24000
Writing audio file with parameters:
Channels: 1
Sample rate: 24000
Sample width: 2
Data length: 1636366 bytes


In [35]:
display(Audio(file_name))

### 相關注意事項 :

- 支援的模型：
  - Gemini 2.5 Flash Preview TTS (支援單人與多人)
  - Gemini 2.5 Pro Preview TTS (支援單人與多人)
- 語音選項：
  - 提供 30 種 預設語音選項，風格各異，例如：Kore (堅定) ,Puck (樂觀) ,Charon (資訊豐富) ,Enceladus (有氣息感)
- 支援的語言：
  - 模型會自動偵測輸入語言，支援 24 種 語言，包含：
    - 英文 (en-US)、日文 (ja-JP)、韓文 (ko-KR)、法文 (fr-FR)、西班牙文 (es-US) 等
    - 中文 (zh-CN) (目前文件上沒有，但使用上可以，可以作為早期測試)
- 主要限制：
  - 模型僅接受純文字輸入，並僅生成純音訊輸出
  - 上下文長度限制為 32k tokens
  - 語言支援僅限於列表中的 24 種
